In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

# Set Param

In [2]:
param = {
    "x_features" : 6, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

args = {
    "output_folder_name" : "./ada_rnn_model",
    "save_model_name" : "test_adarnn_model_by_origin_data1",
    "lr" : 5e-4,
    "n_epochs" : 200,
    "num_layers" : 2, 
    "len_seq" : 24,
    "pre_epoch" : 40,
    "len_win" : 0,
    "dw" : 0.5,
    "early_stop" :40
}
k = 2

# get Data

In [3]:
# read original data
data_pickle = pd.read_pickle("original_data.pkl")
original_data = data_pickle["Dongsi"]

In [4]:
len(original_data), original_data[0].shape, original_data[2].shape

(3, (1460, 24, 6), (1460,))

In [5]:
# original data 에서 data_x, data_y 나누기
data_x = original_data[0]
data_y = original_data[2]

In [6]:
# set time
original_data_start_time = datetime.datetime(2013, 3, 1, 0, 0)

## train data time
train_start_time = '2013-03-01 00:00:00'
train_end_time = '2016-06-30 23:00:00'

## valid data time
valid_start_time='2016-7-2 00:00:00'
valid_end_time='2016-10-30 23:00:00'

In [7]:
# split train & validation data -> get train_x, train_y, valid_x, valid_y
index_start=(pd.to_datetime(train_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(train_end_time) - original_data_start_time).days

train_x=data_x[index_start: index_end + 1]
train_y=data_y[index_start: index_end + 1]

index_start=(pd.to_datetime(valid_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(valid_end_time) - original_data_start_time).days

valid_x=data_x[index_start: index_end + 1]
valid_y=data_y[index_start: index_end + 1]

In [8]:
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

((1218, 24, 6), (1218,), (121, 24, 6), (121,))

In [9]:
train_y

array([0.0013624 , 0.15258856, 0.08855586, ..., 0.14986376, 0.14032698,
       0.        ])

# Data Loader

In [10]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'x_features': 6, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [11]:
train_loader_list, valid_loader = adarnn.create_trainloader(train_x, train_y, valid_x, valid_y, k, train_start_time, train_end_time)

# Training

In [12]:
## test loader 제거 작업 시작
adarnn.train(train_loader_list, valid_loader, args)

[2023-03-17 09:35:39] - create model...
[2023-03-17 09:35:39] - Epoch: 0
[2023-03-17 09:35:39] - training...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  4.42it/s]

[2023-03-17 09:35:40] - evaluating...



Valid: 100%|█████████████████████████████████████| 2/2 [00:00<00:00, 197.09it/s]

tensor([0.0177, 0.0531, 0.0518, 0.2139, 0.1499, 0.0232, 0.2493, 0.0409, 0.0490,
        0.0177, 0.0722, 0.0817, 0.1362, 0.1580, 0.1376, 0.1308, 0.0381, 0.1008,
        0.1907, 0.0858, 0.1744, 0.0668, 0.0599, 0.0586, 0.0899, 0.0436, 0.3324,
        0.1730, 0.0054, 0.0300, 0.1975, 0.1907, 0.2452, 0.1376, 0.1662, 0.0191,
        0.0341, 0.1444, 0.0409, 0.1431, 0.0286, 0.0613, 0.0095, 0.0272, 0.0708,
        0.0450, 0.0913, 0.1403, 0.0259, 0.1431, 0.0559, 0.2589, 0.0531, 0.1662,
        0.0191, 0.1689, 0.1172, 0.0763, 0.0613, 0.4196, 0.3174, 0.0027, 0.1049,
        0.1989], device='cuda:0')
tensor([0.0123, 0.0341, 0.0518, 0.0640, 0.1826, 0.1008, 0.2139, 0.1431, 0.0804,
        0.1757, 0.2084, 0.1226, 0.3992, 0.0463, 0.1635, 0.1158, 0.0477, 0.0272,
        0.0409, 0.1839, 0.0736, 0.0354, 0.0422, 0.1199, 0.0613, 0.2153, 0.1798,
        0.0504, 0.1308, 0.0668, 0.0109, 0.0749, 0.2929, 0.2398, 0.0041, 0.1662,
        0.0872, 0.0913, 0.0640, 0.1131, 0.1008, 0.0068, 0.0749, 0.0668, 0.0586,
      

...

tensor([0.1253, 0.0572, 0.2929, 0.0381, 0.2493, 0.0041, 0.1049, 0.0095, 0.0668,
        0.0613, 0.0054, 0.0749, 0.0341, 0.1308, 0.3174, 0.2398, 0.1008, 0.1362,
        0.1172, 0.1662, 0.0463, 0.2589, 0.0422, 0.0586, 0.0872, 0.2139, 0.0913,
        0.1798, 0.3992, 0.0354, 0.0599, 0.1580, 0.1131, 0.0354, 0.3324, 0.0286,
        0.0736, 0.0708, 0.1989, 0.0436, 0.0191, 0.0749, 0.0640, 0.0817, 0.1907,
        0.2084, 0.0668, 0.1757, 0.0804, 0.2139, 0.0477, 0.1839, 0.0559, 0.0531,
        0.0858, 0.2153, 0.1431, 0.0531, 0.0177, 0.0504, 0.0395, 0.1975, 0.0300,
        0.1826], device='cuda:0')
tensor([0.1008, 0.0272, 0.0109, 0.0218, 0.1744, 0.0409, 0.0191, 0.0817, 0.0272,
        0.1308, 0.0409, 0.0586, 0.1635, 0.1008, 0.1662, 0.1689, 0.0054, 0.0068,
        0.0613, 0.1376, 0.1199, 0.0341, 0.2452, 0.0436, 0.1158, 0.0259, 0.0450,
        0.0409, 0.0518, 0.1730, 0.1907, 0.0232, 0.0777, 0.1376, 0.0899, 0.0177,
        0.0177, 0.1431, 0.1403, 0.1199, 0.0763, 0.1444, 0.4196, 0.1662, 0.0640,
      


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 20.86it/s]

[2023-03-17 09:35:55] - evaluating...



Valid: 100%|█████████████████████████████████████| 2/2 [00:00<00:00, 221.68it/s]

tensor([0.0218, 0.0300, 0.0668, 0.1839, 0.1989, 0.4196, 0.1308, 0.0341, 0.0027,
        0.0504, 0.0736, 0.0381, 0.1431, 0.1403, 0.1635, 0.0259, 0.1172, 0.1376,
        0.1744, 0.0409, 0.1689, 0.1907, 0.0191, 0.1826, 0.0586, 0.3174, 0.0899,
        0.0177, 0.1975, 0.0054, 0.0409, 0.1226, 0.1730, 0.0586, 0.1662, 0.0436,
        0.2929, 0.0422, 0.0913, 0.0518, 0.0450, 0.1662, 0.0817, 0.0777, 0.0191,
        0.0913, 0.0763, 0.0531, 0.1444, 0.0395, 0.0668, 0.1199, 0.0054, 0.0817,
        0.0354, 0.1008, 0.2153, 0.0490, 0.1308, 0.1431, 0.1757, 0.0518, 0.1008,
        0.3324], device='cuda:0')
tensor([0.2452, 0.0272, 0.1798, 0.0286, 0.0177, 0.2589, 0.1158, 0.0668, 0.0354,
        0.0559, 0.0109, 0.0068, 0.0272, 0.0749, 0.2139, 0.0531, 0.0613, 0.0858,
        0.0708, 0.0095, 0.1580, 0.2139, 0.0599, 0.0341, 0.1049, 0.1362, 0.1376,
        0.0572, 0.1199, 0.0123, 0.0722, 0.1253, 0.0749, 0.2084, 0.0477, 0.0872,
        0.1662, 0.0640, 0.2398, 0.0409, 0.0041, 0.0640, 0.0313, 0.0177, 0.0613,
      